In [ ]:
#@title Check GPU availability (should be T4/A100 for fastest training)
import torch
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only'
print('Torch version:', torch.__version__)
print('Device:', gpu_name)
assert torch.cuda.is_available(), '⚠️ Enable GPU in Runtime → Change runtime type → Hardware accelerator: GPU'

## 1. Install dependencies
Installs Ultralytics YOLO and helper libraries.

In [ ]:
!pip install -q ultralytics==8.3.0 roboflow

## 2. (Optional) Mount Google Drive
Mount if your dataset zip lives in Drive or you want to save outputs there.

In [ ]:
from google.colab import drive
DRIVE_MOUNT = False  #@param {type:"boolean"}
if DRIVE_MOUNT:
    drive.mount('/content/drive')

## 3. Upload or reference your dataset zip
- Export your dataset from Roboflow as **YOLOv8** format.
- In Colab, use the folder icon → Upload to place the zip at `/content/floorplan-dataset.zip` (or change the path below).

In [ ]:
DATASET_ZIP_PATH = '/content/floorplan-dataset.zip'  #@param {type:"string"}
DATASET_ROOT = '/content/floorplan-dataset'
import os, zipfile
assert os.path.exists(DATASET_ZIP_PATH), f'Missing dataset zip: {DATASET_ZIP_PATH}'
os.makedirs(DATASET_ROOT, exist_ok=True)
with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(DATASET_ROOT)
print('Dataset extracted to', DATASET_ROOT)

## 4. Train YOLOv8n
Adjust epochs/image size if you want higher accuracy (more time).

In [ ]:
from ultralytics import YOLO
import pathlib
DATA_YAML = next(pathlib.Path(DATASET_ROOT).rglob('data.yaml'))
MODEL_NAME = 'yolov8n.pt'
EPOCHS = 30  #@param {type:"slider", min:10, max:150, step:5}
IMG_SIZE = 416  #@param {type:"slider", min:320, max:640, step:32}
BATCH = 16  #@param {type:"slider", min:8, max:32, step:2}
model = YOLO(MODEL_NAME)
results = model.train(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    patience=5,
    device=0,
    cache=True,
    project='floorplan-runs',
    name='yolov8n-floorplan'
)
results

## 5. Export ONNX
Exports the best checkpoint to ONNX (opset 12 for ONNXRuntime compatibility).

In [ ]:
best_model = YOLO('floorplan-runs/yolov8n-floorplan/weights/best.pt')
onnx_path = best_model.export(format='onnx', opset=12, simplify=True)
print('ONNX saved to', onnx_path)

## 6. Download artifacts
Download the ONNX model (and optional metrics) to your local machine.

In [ ]:
from google.colab import files
files.download(onnx_path)
# Optional: download training results
files.download('floorplan-runs/yolov8n-floorplan/results.csv')